# Clarify Playground

## Getting started
To connect with your Clarify account you need to add your credentials to the workspace. See the [docs](https://docs.clarify.us/docs/obtaining-an-access-token).

In [ ]:
# import credentials
import json
f = open('clarify-credentials.json',)
credentials = json.load(f)
f.close()

In [ ]:
# get access token
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

session = OAuth2Session(client = BackendApplicationClient(client_id=credentials["credentials"]["clientId"]))

try:
    session.fetch_token(
        body=f'audience={credentials["apiUrl"]}', 
        token_url="https://login.clarify.us/oauth/token", 
        client_secret=credentials["credentials"]["clientSecret"]
    )
except:
    raise Exception('failed to get access token: {}'.format(format_exc()))

token = session.access_token

# NB! Does not produce valid token for data retrieval until Public API is finished.

In [ ]:
# get data 
from jsonrpcclient.clients.http_client import HTTPClient
from jsonrpcclient.requests import Request

url_endpoint = "https://clarifyapp.clarify.us/api/timeseries/rpc" # api endpoint
method = "timeseries.Floats" # rpc method call
item_id = "" # id of item (found in item view)

client = HTTPClient(url_endpoint)
headers = {
    "content-type": "application/json",
    "authorization": "Bearer " + token
}

client.session.headers.update(headers)

rpc_text = Request(
    method,
    query = {
        "id": item_id
    },
    window = {
        "from": "2017-01-20T00:00:00Z",
        "to": "2021-12-21T00:00:00Z"
    }
)

res = client.send(rpc_text)
result = res.data.result

In [ ]:
import pandas as pd
import numpy as np

# reformat data
data = []
for section in result["sections"]:
    data += section

ts = np.array(data)
time, value = pd.to_datetime(ts[:,0]), pd.to_numeric(ts[:,1])
series = pd.Series(value, index=time)
print(series.head(5))
series.plot();